In [ ]:
import joblib
import pandas as pd


# 选择需要归一化的列
columns_to_normalize = ['time_order', 'provincial_load_forecast', 'tie_line_load_forecast',
                        'total_power_forecast', 'new_energy_power_forecast',
                        'hydraulic_power_forecast', 'non_market_power_forecast',
                        'dayahead_clearing_price']
# # 加载 scaler
scaler = joblib.load('../data/scaler.joblib')
data_train = pd.read_feather('../data/train_data.feather')
data_val = pd.read_feather('../data/val_data.feather')
data_test = pd.read_feather('../data/test_data.feather')
# 对上面的数据集进行归一化
data_train[columns_to_normalize] = scaler.transform(data_train[columns_to_normalize])
data_val[columns_to_normalize] = scaler.transform(data_val[columns_to_normalize])
data_test[columns_to_normalize] = scaler.transform(data_test[columns_to_normalize])

